In [6]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)

### Enlarge the Xtrain matrix with ones ###
[samples, _]=housing.data.shape
dataset = np.concatenate((housing.data, np.ones((samples, 1))), axis = 1)

### Split training and testing dataset ###
split_index = int(0.9*samples)
training = dataset[0:split_index, :]
testing = dataset[split_index:, :]

target = housing.target[:, np.newaxis]
training_target = target[0:split_index, :]
testing_target = target[split_index:, :]
#train_mean = np.mean(training, axis=0)
#training = training - train_mean


###### Implement Data Preprocess here ######




##### Derive weight from normal equation #####
x_train = tf.convert_to_tensor(training)
y_train = tf.convert_to_tensor(target)
x_trans = tf.transpose(x_train)
w = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(x_trans, x_train)),x_trans),y_train)
y_pred = tf.matmul(x_train, w)

###### Start TF session ######
with tf.Session() as sess:
    print("Hello")
    error_rate = tf.reduce_mean(tf.reduce_sum(tf.divide((y_train-y_pred),y_train), reduction_indices=[1]))
    print(sess.run(error_rate))

    #show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######

Shape of dataset: (20640, 8)
Shape of label: (20640,)
Hello
-0.123676147704
